In this notebook, we begin our investigation of the **Si**nusoidal **Re**presentation **N**etworks (SIREN)  presented in __*Sitzmann, V., Martel, J., Bergman, A., Lindell, D., & Wetzstein, G. (2020). Implicit neural representations with periodic activation functions. Advances in Neural Information Processing Systems, 33, 7462-7473*.__

The intention here is to work our way to neural fields, and hopefully to some PDE applications. 


As a learning excersize, we first implement a simple network using Micrograd.


#Micrograd implementation

*This has been copied over from another notebook, which was based on Karpathy's tutorial.


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
# first we build a data structure to keep track of operations
class Value:

  def __init__(self, data, _children=(), _op='',label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None 
    self._prev = set(_children) #in the forward mode, information propagates from children to parents
    self._op = _op
    self.label = label

  def __repr__(self):
    #return f"Value(data={self.data}, label={self.label})"
    return f"Value(data={self.data})"

  def __neg__(self): #-self
    return self * -1

  def __add__(self, other): #order: self + other
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data + other.data, (self, other), _op='+')
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    return out

  def __radd__(self, other): #order: other + self
    return self + other

  def __sub__(self, other): #order: self - other
    return self + (-other)

  def __mul__(self, other): #order: self * other
    other = other if isinstance(other,Value) else Value(other)
    out = Value(self.data * other.data, (self, other), _op='*')
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out
  
  def __pow__(self,deg):
    #assuming other is a constant and not a variable
    assert(isinstance(deg, (int, float)), "only supports powers that are int or float")
    out = Value(self.data ** deg, (self,), _op=f'**{str(deg)}')
    def _backward():
      self.grad += deg * self.data ** (deg-1)  * out.grad
    out._backward = _backward
    return out

  def __truediv__(self, other): #order: self / other
    return self * other**-1

  def __rmul__(self,other): # order: other * self (reversed order)
    return self*other

  def relu(self):
    x = self.data
    t = x if x > 0 else 0
    out = Value(t, (self,), _op = 'relu')
    def _backward():
      self.grad += 1.0 * out.grad if x > 0 else 0
    out._backward = _backward
    return out

  def tanh(self):
    n = self.data
    t = (math.exp(2*n) - 1)/(math.exp(2*n)+1)
    out = Value(t, (self,), _op='tanh')
    def _backward():
      self.grad += (1-t**2) * out.grad
    out._backward = _backward
    return out

  def sigmoid(self):
    x = self.data
    t = 1 / 1-math.exp(-x)
    out = Value(t, (self,), _op='sigm')
    def _backward():
      self.grad += t * (1-t) * out.grad
    out._backward = _backward

  def sin(self):
    x = self.data
    t = math.sin(x)
    out = Value(t, (self,), _op='sin')
    def _backward():
      self.grad += math.cos(x) * out.grad
    out._backward = _backward
    return out

  def cos(self):
    x = self.data
    t = math.cos(x)
    out = Value(t, (self,), _op='cos')
    def _backward():
      self.grad += -math.sin(x) * out.grad
    out._backward = _backward
    return out

  def exp(self):
    x = self.data
    t = math.exp(x)
    out = Value(t, (self,), _op='exp')
    def _backward():
      self.grad += t * out.grad
    out._backward = _backward
    return out
    
  def backward(self):
    # build topological order
    topo=[]
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev: # all children are added recursively to topo before the parent (v)
          build_topo(child)
        topo.append(v) #append parent after all children
    build_topo(self)
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

  def zero_grad(self):
    self.grad = 0

In [ ]:
# visualization
from graphviz import Digraph

def trace(root): #builds a graph from a Value object
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child,v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root): #visualizes graph from value object
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) #LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))

    dot.node(name = uid, label="{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape = 'record')
    if n._op:
      dot.node(name=uid+n._op, label = n._op)
      dot.edge(uid + n._op, uid)
  for n1,n2 in edges:
    dot.edge(str(id(n1)), str(id(n2))+n2._op)
  
  return dot

In [ ]:
#Simple example

x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

b = Value(6.7, label='b')

x1w1 = x1*w1; x1w1.label = 'x1*w1' 
x2w2 = x2*w2; x2w2.label='x2*w2'

x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1+x2*w2'
n = x1w1x2w2+b; n.label='n'
o = n.relu(); o.label='o'
draw_dot(o)

In [ ]:
o.backward()
draw_dot(o)

In [ ]:
class Neuron:

  def __init__(self, nin):
    self.w = [Value(np.random.uniform(-1,1), label='w') for i in range(nin)]
    self.b = Value(np.random.uniform(-1,1), label='b')
    self.preact = None

  def __call__(self,x):
    #w*x+b
    out = sum((wi*xi for wi,xi in zip(self.w,x)), self.b)
    self.preact = out
    return out
  
  def parameters(self):
    return self.w + [self.b] 

class Layer:

  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for i in range(nout)]
    self.outs = None

  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    self.outs = outs
    return outs #outs[0] if len(outs)==1 else outs

  def parameters(self):
     return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

  def __init__(self, nin, nouts):
    sz = [nin] + nouts #prepends nin to list of nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

  def __call__(self,x):
    for layer in self.layers[:-1]: #forward pass through the MLP
      x = layer(x)
      x = [t.sin() for t in x]
    x = self.layers[-1](x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [ ]:
# training data
dx = 0.05
xs = [i*dx for i in range(100)]
ys = [1.0 * math.sin(3*x) + x**2 - 5*x for x in xs]

num_train = len(xs)

In [ ]:
n = MLP(1, [20,20,1])

#before training
ypred = [n([x])[0].data for x in xs]
plt.plot(xs,ys, 'bo', label = 'training')
plt.plot(xs,ypred, 'r', label='model (pre-training)')
plt.show()

In [ ]:
# for debugging, looking at pre-activations
# d=0
# num_neurons = len(n.layers[d].neurons)
# preacts = n.layers[d].outs 
# preacts

In [ ]:
params = [p.data for p in n.parameters()]
plt.hist(params)
plt.show()

In [ ]:
for k in range(100):
  
  # forward pass
  ypred = [n([x])[0] for x in xs]
  loss = 1/num_train * sum((yout-ygt)**2 for ygt, yout in zip(ys,ypred))

  #backward pass
  for p in n.parameters(): #recall gradients accumulate in the backward pass, so we need to zero out things beforehand
    p.grad = 0.0
  loss.backward()

  #update
  for p in n.parameters(): 
    p.data += -0.1 * p.grad

  print(f'step {k} | loss: {loss.data}')

In [ ]:
params = [p.data for p in n.parameters()]
plt.hist(params)
plt.show()

In [ ]:
#after training
ypred = [n([x])[0].data for x in xs]
plt.plot(xs,ys, 'bo', label = 'training')
plt.plot(xs,ypred, 'r', label='model (pre-training)')
plt.legend()
plt.show()
